# Callbacks

Quoting from the Gurobi [documentation](https://www.gurobi.com/documentation/8.1/refman/py_callbacks.html)

> A callback is a **user function** that is **called periodically** by the Gurobi optimizer in order to allow the user to **query or modify** the state of the optimization.

That is, we create our own function which we will call *the callback* and we pass it to the ``optimie`` method. The callback will then be called at specific points while the problem is being solved. We decide at which points the callback should be called, and within the callback we can use specific methods to query details on the state of the optimization procedure, and we can also modify what the optimization algorithm is doing. ¨

Let us build and example MIP to which we will attach a callback.

In [17]:
# We import the necessary stuff and build a toy MIP
from gurobipy.gurobipy import Model, GRB, Column
import random as r

def build_model():
    
    # Sets the seed of the random generator in 
    # order to be able to replicate the results
    r.seed(1)
    
    # Creates the model
    m = Model()

    # Creates 300 integer variables. 
    x = m.addVars(300, vtype=GRB.INTEGER, name="x")
    
    # The variables should be appended as attributes 
    # with a leading _ to the model object in order 
    # to be visible in the callback.
    m._x = x

    # Creates the objective with random costs between 10 and 30
    expr = x.prod({i:(10 + r.random()*20) for i in range(300)})
    m.setObjective(expr, GRB.MINIMIZE)

    # Creates a constraint with random coefficients between -20 and +20
    lhs1 = x.prod({i:(-20 + r.random()*40) for i in range(300)})
    c1 = m.addConstr(lhs1, GRB.GREATER_EQUAL, 120, "c1")

    # Creates a constraints with random coefficients between -5 and 5
    lhs2 = x.prod({i:(-5 + r.random()*10) for i in range(300)})
    c2 = m.addConstr(lhs2, GRB.GREATER_EQUAL, 70, "c2")
    
    # Finally it retuns the model
    return m

A callback is created by defining a function. The function takes as argument the model where we want to use the callback and a ``where`` variable. The where variable is used to check whether the optimizer reached the intended point during the execution. The basic structure of a callback is the following

In [ ]:
# Basic structure of a callback
def my_callback(model, where):
    
    # 1. Check if we are at the intended where
    
    # 2. Execute the code of the callback 
    # (e.g., query details or modify execution)

To tell Gurobi that we want to use the callback then we simply pass the callback to the ``optimize`` method as follows

In [ ]:
m.optimize(my_callback)

Now, during the course of the optimization, Gurobi will periodically call our function ``my_callback`` passing as arguments the focal ``model`` (``m`` in our case) and a ``where``. The ``where`` takes a value which corresponds to a different stage in the course of the algorithm used to solve the problem. The possible values are listed at the [Callback Coded page](https://www.gurobi.com/documentation/8.1/refman/callback_codes.html#sec:CallbackCodes). As an example, if Gurobi is currently executing a presolve it will pass a value of ``1`` (corresponding to ``GRB.Callback.PRESOLVE`` in Python) for the ``where``, and if it has just found an integer solution during the course of the Branch & Bound method it will pass a value of ``4`` (corresponding to ``GRB.Callback.MIPSOL`` in Python). 
In this way the callback is able to understand at what stage of the algorithm the solver is, and execute accordingly.

As an example, let us pass a very simple callback to our solver which simply gives us basic information.

In [18]:
# We build a model using the function we created above
m = build_model()

# We create
def my_callback(model,where):
    if where == GRB.Callback.MIPNODE:
        print(">>> We are at a B&C node")
    if where == GRB.Callback.MIPSOL:
        print(">>> We found an integer solution!")

# We tell Gurobi not to print the default output
# so that we can isolate our output more easily
m.setParam(GRB.Param.OutputFlag,0) 
m.optimize(my_callback)

>>> We found an integer solution!
>>> We found an integer solution!
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We 

Let us now try to get some more useful information about what is happening during the solution of our problem.
In order to retrieve information during the execution of an algorithm (e.g., Branch & Bound), a Model object
has a few methods: 
+ [``cbGet``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbget.html), which can be used to query one of the possible ``what`` information listed [here](https://www.gurobi.com/documentation/8.1/refman/callback_codes.html#sec:CallbackCodes).
+ [``cbGetSolution``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbgetsolution.html), which can be used only when ``where==GRB.Callback.MIPSOL`` to retrieve the newly found integer solution
+ [``cbGetNodeRel``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbgetnoderel.html), which can be used only when ``where==GRB.Callback.MIPNODE`` and ``GRB.Callback.MIPNODE_STATUS==GRB.OPTIMAL`` to retrieve the value of the node relaxation solution at the current node.

Let us see some examples.

In [19]:
# We re-build a model using the function we created above
m = build_model()

# We create our callback
def my_callback(model,where):
    
    if where == GRB.Callback.MIPNODE:
        print(">>> We are at B&C node ",model.cbGet(GRB.Callback.MIPNODE_NODCNT))
        print("Status ",model.cbGet(GRB.Callback.MIPNODE_STATUS))
        # Status codes are reported here https://www.gurobi.com/documentation/8.1/refman/optimization_status_codes.html#sec:StatusCodes
        print("Best objective ", model.cbGet(GRB.Callback.MIPNODE_OBJBST))
        print("Best bound ", model.cbGet(GRB.Callback.MIPNODE_OBJBND))
    
    if where == GRB.Callback.MIPSOL:    
        # If we find an integer solution we print its value.
        # Remember that when we build the model we create the attribute
        # _x to our model object in order to be able to access the solution.
        print(">>> New integer solution!")
        for i in range(300):
            if model.cbGetSolution(model._x[i]) > 0:
                print("x[%d]: %f " % (i,model.cbGetSolution(model._x[i])))
        

m.setParam(GRB.Param.OutputFlag,0) # We tell Gurobi not to print the default output
m.optimize(my_callback)

>>> New integer solution!
x[299]: 42.000000 
>>> New integer solution!
x[91]: 1.000000 
x[281]: 15.000000 
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  155.85456572544138
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  156.1850685876831
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  158.52942228414668
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  159.19229747204184
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  159.68656909581023
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  159.74240365603987
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  160.0967536635286
>>> We are at B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  160.1265439637806
>>> We are at B&C node  0.0
Status  2
Best objective  163.266028

One of the useful things we can do in a call back is adding *lazy constraints*. Lazy constraints can be thought of as part of the original model which we wish to add only if we find them violated. We usually treat as lazy constraints  
+ those constraints of which we have too many to build a manageable model (think for example of the subtour elimination constraints in the TSP)
+ those constraints which, on top of being numerous, are too hard explicitly formulate upfront (the perfect example are the constraints of the Benders reformulation: if we were to add them upfront we would have to enumerate the extreme points of the subproblems)
In these cases we prefer to add them lazily, that is, only once we find the violated. 

Lazy constraints are added, within a callback, with the method [``cbLazy``](https://www.gurobi.com/documentation/8.1/refman/py_model_cblazy.html). This method can be called only when ``where`` is either ``GRB.Callback.MIPNODE`` or ``GRB.Callback.MIPSOL``.

Let us see an extremely simple example which will help us to understand what happens when we add lazy constraints. Let us suppose that in the example model provided above, there should be a constraint that says that the sum of the first 50 decision variables should be positive. Assume that, for some reason, we treat this constraint as *lazy* and, as such, the constraint is not part of the initial formulation. In the forme code sample we saw that, without that constraint, the model finds two integer solutions. Both violate the lazy constraint. Let us now add a callback 
1. checks whether $\sum_{i=1}^{50}x_i > 0$
2. if $\sum_{i=1}^{50}x_i \leq 0$ adds a lazy constraint $\sum_{i=1}^50x_i \geq 1$


In [22]:
# We re-build a model using the function we created above
m = build_model()

# We create our callback
def my_callback(model,where):

    if where == GRB.Callback.MIPSOL:    
        # If we find an integer solution we print its value.
        print("============================================================")
        print(">>> New integer solution at node ", model.cbGet(GRB.Callback.MIPSOL_NODCNT))
        for i in range(300):
            if model.cbGetSolution(model._x[i]) > 0:
                print("x[%d]: %f " % (i,model.cbGetSolution(model._x[i])))

        # Then we check if the sum of the fist 50 variables is positive
        sum_50 = 0
        for i in range(50):
            sum_50 = sum_50 + model.cbGetSolution(model._x[i])
        print("The sum of the first 50 variables is ",sum_50)
        if sum_50 == 0:
            # In this case we create a lazy constraint
            lhs = 0
            for i in range(50):
                lhs = lhs + model._x[i]
            model.cbLazy(lhs >= 1)
            print("Added a lazy constraint")
        else:
            print("The solution respects the lazy constraint")
                
# We need to inform Gurobi that we might add lazy constraints
m.setParam(GRB.Param.LazyConstraints, 1)
m.setParam(GRB.Param.OutputFlag,0) # We tell Gurobi not to print the default output
m.optimize(my_callback)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
>>> New integer solution at node  0.0
x[299]: 42.000000 
The sum of the first 50 variables is  0.0
Added a lazy constraint
>>> New integer solution at node  0.0
x[299]: 42.000000 
The sum of the first 50 variables is  0.0
Added a lazy constraint
>>> New integer solution at node  0.0
x[281]: 16.000000 
The sum of the first 50 variables is  0.0
Added a lazy constraint
>>> New integer solution at node  0.0
x[35]: 1.000000 
x[281]: 15.000000 
The sum of the first 50 variables is  1.0
The solution respects the lazy constraint
>>> New integer solution at node  0.0
x[35]: 1.000000 
x[281]: 15.000000 
The sum of the first 50 variables is  1.0
The solution respects the lazy constraint
>>> New integer solution at node  0.0
x[35]: 1.000000 
x[281]: 15.000000 
The sum of the first 50 variables is  1.0
The solution respects the lazy constraint


Finally, note that Gurobi also offers a method [``cbCut``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbcut.html) which might well be confused with ``cbLazy``. The method ``cbCut`` is used to add *cutting planes* to strengthen the formulation of the LP relaxation and obtain a tighter bound. The method ``cbLazy`` is instead used to add missing constraints from the formulation. While the former are *valid inequalities* which are satisfied by any integer solution in the initial formulation (i.e., cutting planes are not supposed to cut off integer solutions), the latter are instead meant to cut off integer solutions which violate the inequality. In the former case, inequalities are derived from the existing constraints. In the latter, constraints are missing from the current pool of constraints. An example of the former type of cuts is a *Gomory cut*, while an example of the latter is a *subtour elimination constraint*. The former strengthens the formulation, the latter cuts off infeasible integer solutions.